# PyTorch Experiments to predict Grit Mindset

## Import Libraries

In [2]:
# install skorch, a scikilt learn wrapper for PyTorch
! pip install -U skorch

     |████████████████████████████████| 102kB 592kB/s ta 0:00:01


In [22]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
import os
import sys
import glob
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.metrics import accuracy_score, precision_score, recall_score
import matplotlib.pyplot as plt
from skorch import NeuralNetClassifier
from sklearn.model_selection import GridSearchCV

### Note Notebook doesn't display all the rows and columns
- let's fix that

In [5]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

## Check CSV file inside data folder

In [6]:
! ls data

GritMindset.csv                      models
GritMindset.csvDroppedNaNCols.pickle


In [7]:
def loadData(csvFile):
    pickleDump = '{}DroppedNaNCols.pickle'.format(csvFile)
    if os.path.exists(pickleDump):
        df = pd.read_pickle(pickleDump)
    else:
        df = pd.read_csv(csvFile, low_memory=False)
        # clean data
        # strip the whitspaces from column names
        df = df.rename(str.strip, axis='columns')
        # drop Infinity rows and NaN string from each column
        for col in df.columns:
            indexNames = df[df[col] == 'Infinity'].index
            if not indexNames.empty:
                print('deleting {} rows with Infinity in column {}'.format(len(indexNames), col))
                df.drop(indexNames, inplace=True)
            
        # drop all columns with NaN values
        beforeColumns = df.shape[1]
        df.dropna(axis='columns', inplace=True)
        print('Dropped {} columns with NaN values'.format(beforeColumns - df.shape[1]))
        # drop all rows with NaN values
        beforeRows = df.shape[0]
        df.dropna(inplace=True)
        print('Dropped {} rows with NaN values'.format(beforeRows - df.shape[0]))
        df.to_pickle(pickleDump)
        
    return df

In [8]:
df = loadData('data/GritMindset.csv')

In [9]:
# let's check the shape again
df.shape

(309, 4)

In [10]:
# display first 5 samples
df.head()

,Gender,Age,HighSchoolGrade,HonorsScience
0,1,19,85.0,2
1,2,19,94.0,1
2,1,19,85.0,2
3,2,19,80.0,2
4,2,18,78.0,2


In [11]:
df.columns

Index(['Gender', 'Age', 'HighSchoolGrade', 'HonorsScience'], dtype='object')

In [12]:
# class distribution
label = 'HonorsScience'
print(df.groupby(label).size())

HonorsScience
1     48
2    261
dtype: int64


## Binary-class classification
- Relabel HonorsScience 1 as 0 and 2 as 1

In [13]:
lblTypes = set(df[label])
lblTypes

{1, 2}

In [14]:
lblTypes = dict(zip(lblTypes, [0]*2))
lblTypes[2] = 1
lblTypes

{1: 0, 2: 1}

In [15]:
df[label] = df[label].map(lblTypes)

In [16]:
df.head()

,Gender,Age,HighSchoolGrade,HonorsScience
0,1,19,85.0,1
1,2,19,94.0,0
2,1,19,85.0,1
3,2,19,80.0,1
4,2,18,78.0,1


In [17]:
sss = StratifiedShuffleSplit(n_splits = 10, test_size=0.2, random_state=0)
print(sss)

StratifiedShuffleSplit(n_splits=10, random_state=0, test_size=0.2,
            train_size=None)


In [48]:
class Net(nn.Module):
    def __init__(self):
        # define nn
        super().__init__()
        self.inputLayer = nn.Linear(3, 50)
        self.relu1 = nn.ReLU()
        #self.dout = nn.Dropout(0.2)
        self.layer2 = nn.Linear(50, 100)
        self.prelu = nn.PReLU(1)
        self.out = nn.Linear(100, 2)
        self.out_act = nn.Sigmoid()
    
    def forward(self, X):
        X = self.inputLayer(X)
        X = self.relu1(X)
        #X = self.dout(X)
        X = self.layer2(X)
        X = self.prelu(X)
        X = self.out(X)
        y = self.out_act(X)
        return y
    

In [49]:
"""
model = Net()
opt = optim.Adam(net.parameters(), lr=0.001, betas=(0.9, 0.999))
criterion = nn.BCELoss()
"""
model = NeuralNetClassifier(Net, max_epochs=100, lr=0.1, iterator_train__shuffle=True,)

In [ ]:
data_y = df.pop("HonorsScience").values
data_X = df.values

In [50]:
data_X = data_X.astype(np.float32)
data_y = data_y.astype(np.int64)
params = {
    'lr': [0.01, 0.02],
    'max_epochs': [10, 100],
    'module_num_units': [10,20],
}

gs = GridSearchCV(model, params, refit=False, cv=10, scoring='accuracy')

gs.fit(data_X, data_y)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        3.4075       0.8393        0.0030  0.0219
      2        0.0039       0.8393        0.0017  0.0113
      3        0.0027       0.8393        0.0012  0.0107
      4        0.0022       0.8393        0.0009  0.0132
      5        0.0019       0.8393        0.0007  0.0117
      6        0.0017       0.8393        0.0006  0.0111
      7        0.0016       0.8393        0.0005  0.0113
      8        0.0014       0.8393        0.0004  0.0116
      9        0.0013       0.8393        0.0004  0.0128
     10        0.0012       0.8393        0.0004  0.0112
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.0115       0.8393        0.0100  0.0080
      2        0.0067       0.8393        0.0083  0.0124
      3        0.0053       0.8393        0.0074  0.0105
      4        0.0045       0.8

      4        0.0000       0.8393        0.0000  0.0113
      5        0.0000       0.8393        0.0000  0.0106
      6        0.0000       0.8393        0.0000  0.0103
      7        0.0000       0.8393        0.0000  0.0099
      8        0.0000       0.8393        0.0000  0.0102
      9        0.0000       0.8393        0.0000  0.0101
     10        0.0000       0.8393        0.0000  0.0119
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.0636       0.8393        0.0014  0.0090
      2        0.0044       0.8393        0.0010  0.0108
      3        0.0039       0.8393        0.0008  0.0103
      4        0.0036       0.8393        0.0007  0.0101
      5        0.0034       0.8393        0.0006  0.0108
      6        0.0032       0.8393        0.0005  0.0102
      7        0.0030       0.8393        0.0005  0.0105
      8        0.0029       0.8393        0.0004  0.0112
      9        0.0028       0.8

      9        0.0027       0.8393        0.0145  0.0124
     10        0.0026       0.8393        0.0143  0.0106
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.1006       0.8393        0.0012  0.0109
      2        0.0004       0.8393        0.0012  0.0114
      3        0.0004       0.8393        0.0012  0.0122
      4        0.0003       0.8393        0.0012  0.0110
      5        0.0003       0.8393        0.0011  0.0116
      6        0.0003       0.8393        0.0011  0.0108
      7        0.0003       0.8393        0.0011  0.0120
      8        0.0003       0.8393        0.0011  0.0112
      9        0.0003       0.8393        0.0011  0.0105
     10        0.0003       0.8393        0.0011  0.0114
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        1.6888       0.8393        0.0001  0.0081
      2        0.0004       0.8

     95        0.0002       0.8393        0.0000  0.0102
     96        0.0002       0.8393        0.0000  0.0114
     97        0.0002       0.8393        0.0000  0.0106
     98        0.0002       0.8393        0.0000  0.0118
     99        0.0001       0.8393        0.0000  0.0109
    100        0.0001       0.8393        0.0000  0.0102
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.0212       0.7857        0.0169  0.0111
      2        0.0033       0.7857        0.0160  0.0122
      3        0.0030       0.7857        0.0153  0.0131
      4        0.0028       0.7857        0.0146  0.0107
      5        0.0026       0.7857        0.0141  0.0101
      6        0.0025       0.7857        0.0135  0.0116
      7        0.0023       0.7857        0.0130  0.0104
      8        0.0022       0.7857        0.0126  0.0110
      9        0.0021       0.7857        0.0122  0.0120
     10        0.0020       0.7

      2        0.0054       0.8393        0.0001  0.0115
      3        0.0049       0.8393        0.0001  0.0110
      4        0.0044       0.8393        0.0001  0.0109
      5        0.0039       0.8393        0.0001  0.0107
      6        0.0036       0.8393        0.0001  0.0104
      7        0.0031       0.8393        0.0001  0.0104
      8        0.0028       0.8393        0.0001  0.0112
      9        0.0025       0.8393        0.0000  0.0110
     10        0.0023       0.8393        0.0000  0.0116
     11        0.0021       0.8393        0.0000  0.0106
     12        0.0019       0.8393        0.0000  0.0100
     13        0.0018       0.8393        0.0000  0.0107
     14        0.0017       0.8393        0.0000  0.0106
     15        0.0015       0.8393        0.0000  0.0102
     16        0.0014       0.8393        0.0000  0.0101
     17        0.0013       0.8393        0.0000  0.0105
     18        0.0012       0.8393        0.0000  0.0103
     19        0.0011       0.8

...

     34        0.0001       0.8393        0.0002  0.0124
     35        0.0001       0.8393        0.0002  0.0107
     36        0.0001       0.8393        0.0002  0.0129
     37        0.0001       0.8393        0.0001  0.0109
     38        0.0001       0.8393        0.0001  0.0112
     39        0.0001       0.8393        0.0001  0.0106
     40        0.0001       0.8393        0.0001  0.0126
     41        0.0001       0.8393        0.0001  0.0107
     42        0.0001       0.8393        0.0001  0.0116
     43        0.0001       0.8393        0.0001  0.0114
     44        0.0001       0.8393        0.0001  0.0106
     45        0.0001       0.8393        0.0001  0.0128
     46        0.0001       0.8393        0.0001  0.0106
     47        0.0001       0.8393        0.0001  0.0104
     48        0.0001       0.8393        0.0001  0.0098
     49        0.0001       0.8393        0.0001  0.0100
     50        0.0001       0.8393        0.0001  0.0106
     51        0.0001       0.8

     42        0.0002       0.7857        0.0016  0.0111
     43        0.0002       0.7857        0.0016  0.0120
     44        0.0002       0.7857        0.0016  0.0103
     45        0.0002       0.7857        0.0016  0.0102
     46        0.0002       0.7857        0.0016  0.0112
     47        0.0002       0.7857        0.0015  0.0111
     48        0.0002       0.7857        0.0015  0.0101
     49        0.0002       0.7857        0.0015  0.0122
     50        0.0002       0.7857        0.0015  0.0104
     51        0.0002       0.7857        0.0015  0.0117
     52        0.0002       0.7857        0.0014  0.0132
     53        0.0002       0.7857        0.0014  0.0118
     54        0.0002       0.7857        0.0014  0.0104
     55        0.0002       0.7857        0.0014  0.0103
     56        0.0002       0.7857        0.0014  0.0103
     57        0.0002       0.7857        0.0014  0.0099
     58        0.0002       0.7857        0.0014  0.0096
     59        0.0002       0.7

     50        0.0003       0.1607        0.0034  0.0125
     51        0.0003       0.1607        0.0034  0.0122
     52        0.0003       0.1607        0.0033  0.0151
     53        0.0003       0.1607        0.0033  0.0126
     54        0.0003       0.1607        0.0032  0.0144
     55        0.0003       0.1607        0.0032  0.0123
     56        0.0003       0.1607        0.0032  0.0123
     57        0.0003       0.1607        0.0031  0.0107
     58        0.0003       0.1607        0.0031  0.0117
     59        0.0003       0.1607        0.0030  0.0117
     60        0.0003       0.1607        0.0030  0.0112
     61        0.0003       0.1607        0.0030  0.0143
     62        0.0003       0.1607        0.0029  0.0150
     63        0.0003       0.1607        0.0029  0.0115
     64        0.0003       0.1607        0.0029  0.0123
     65        0.0003       0.1607        0.0028  0.0119
     66        0.0002       0.1607        0.0028  0.0107
     67        0.0002       0.1

     75        0.0000       0.8393        0.0000  0.0094
     76        0.0000       0.8393        0.0000  0.0096
     77        0.0000       0.8393        0.0000  0.0132
     78        0.0000       0.8393        0.0000  0.0116
     79        0.0000       0.8393        0.0000  0.0116
     80        0.0000       0.8393        0.0000  0.0111
     81        0.0000       0.8393        0.0000  0.0118
     82        0.0000       0.8393        0.0000  0.0106
     83        0.0000       0.8393        0.0000  0.0135
     84        0.0000       0.8393        0.0000  0.0121
     85        0.0000       0.8393        0.0000  0.0127
     86        0.0000       0.8393        0.0000  0.0148
     87        0.0000       0.8393        0.0000  0.0175
     88        0.0000       0.8393        0.0000  0.0134
     89        0.0000       0.8393        0.0000  0.0134
     90        0.0000       0.8393        0.0000  0.0126
     91        0.0000       0.8393        0.0000  0.0209
     92        0.0000       0.8

/Users/rbasnet/miniconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [51]:
# grid search best results
print(gs.best_score_, gs.best_params_)

0.8446601941747572 {'lr': 0.01, 'max_epochs': 100, 'module_num_units': 20}
